# Sentinel-1 SAR Change Detection Analysis
## Hainan Naval Base (Yulin) - August 2025

**Objective**: Detect and analyze changes in infrastructure, ships, and aircraft between August 15 and August 16, 2025.

**Techniques Demonstrated**:
- Multi-temporal SAR image processing
- Change detection (difference and log-ratio methods)
- Object detection (bright spots for ships/planes)
- Professional visualization

---

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

# Import our modules
from src.utils import load_sar_band, get_safe_info
from src.preprocessing import (
    convert_to_db, apply_speckle_filter, enhance_contrast, normalize_intensity
)
from src.change_detection import (
    compute_difference, detect_changes, compute_change_magnitude,
    detect_bright_spots, filter_detections_by_size, refine_change_map
)
from src.visualization import (
    plot_sar_image, plot_before_after, plot_change_heatmap,
    plot_detections, plot_change_rgb
)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ All modules loaded successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Load Multi-Temporal SAR Data

We'll analyze two Sentinel-1 images:
- **Before**: August 15, 2025 (Sentinel-1C)
- **After**: August 16, 2025 (Sentinel-1A)

In [ ]:
# Define paths to .SAFE directories
safe_before = Path("../data/sentinel1/S1C_IW_GRDH_1SDV_20250815T025924_20250815T025949_003677_0075A2_3106.SAFE")
safe_after = Path("../data/sentinel1/S1A_IW_GRDH_1SDV_20250816T025212_20250816T025237_060555_078800_1619.SAFE")

# Get product information
info_before = get_safe_info(safe_before)
info_after = get_safe_info(safe_after)

print("=" * 60)
print("BEFORE Image (August 15, 2025)")
print("=" * 60)
print(f"Satellite: {info_before['satellite']}")
print(f"Product Type: {info_before['product_type']}")
print(f"Polarizations: {info_before['polarizations']}")
print(f"Start Time: {info_before['start_time']}")

print("\n" + "=" * 60)
print("AFTER Image (August 16, 2025)")
print("=" * 60)
print(f"Satellite: {info_after['satellite']}")
print(f"Product Type: {info_after['product_type']}")
print(f"Polarizations: {info_after['polarizations']}")
print(f"Start Time: {info_after['start_time']}")

print("\n⏱ Time Difference: ~1 day")

In [ ]:
# Load VV polarization (better for metal targets like ships and planes)
print("Loading SAR data (VV polarization)...")

vv_before, meta_before = load_sar_band(safe_before, polarization='VV')
vv_after, meta_after = load_sar_band(safe_after, polarization='VV')

print(f"\n✓ Before Image: {vv_before.shape} | dtype: {vv_before.dtype}")
print(f"  Range: {vv_before.min()} - {vv_before.max()} | Mean: {vv_before.mean():.2f}")

print(f"\n✓ After Image: {vv_after.shape} | dtype: {vv_after.dtype}")
print(f"  Range: {vv_after.min()} - {vv_after.max()} | Mean: {vv_after.mean():.2f}")

print(f"\n📊 Total pixels per image: {vv_before.size:,}")
print(f"💾 Memory per image: {vv_before.nbytes / 1024**2:.1f} MB")

## 3. Extract Region of Interest (ROI)

For this analysis, we'll focus on a manageable subset containing the base facilities, runway, and port area.

In [ ]:
# Define ROI (adjust these coordinates to focus on specific areas)
# Full image is 16795 × 25273 pixels
# Let's take a 2000x2000 pixel subset for demonstration

y_start, y_end = 5000, 7000
x_start, x_end = 10000, 12000

roi_before = vv_before[y_start:y_end, x_start:x_end]
roi_after = vv_after[y_start:y_end, x_start:x_end]

print(f"ROI Shape: {roi_before.shape}")
print(f"ROI Coverage: {roi_before.shape[0]} × {roi_before.shape[1]} pixels")
print(f"Approximate Ground Coverage: ~{roi_before.shape[0]*10/1000:.1f}km × {roi_before.shape[1]*10/1000:.1f}km")
print("\n(Sentinel-1 GRD: ~10m spatial resolution)")

## 4. Preprocessing Pipeline

Apply the complete SAR preprocessing chain:
1. Convert to dB scale
2. Apply speckle filter
3. Enhance contrast

In [ ]:
# Convert to dB scale
print("Step 1: Converting to dB scale...")
db_before = convert_to_db(roi_before)
db_after = convert_to_db(roi_after)

print(f"  Before dB range: {db_before.min():.2f} to {db_before.max():.2f} dB")
print(f"  After dB range: {db_after.min():.2f} to {db_after.max():.2f} dB")

# Apply median speckle filter
print("\nStep 2: Applying speckle filter (median, 5×5)...")
filtered_before = apply_speckle_filter(db_before, method='median', size=5)
filtered_after = apply_speckle_filter(db_after, method='median', size=5)

variance_reduction_before = (1 - np.var(filtered_before) / np.var(db_before)) * 100
variance_reduction_after = (1 - np.var(filtered_after) / np.var(db_after)) * 100

print(f"  Variance reduction (before): {variance_reduction_before:.1f}%")
print(f"  Variance reduction (after): {variance_reduction_after:.1f}%")

# Enhance contrast
print("\nStep 3: Enhancing contrast (2-98% stretch)...")
enhanced_before = enhance_contrast(filtered_before, method='stretch')
enhanced_after = enhance_contrast(filtered_after, method='stretch')

print("\n✓ Preprocessing complete!")

## 5. Visualize Preprocessed Images

In [ ]:
# Create before/after comparison
fig, axes = plot_before_after(
    enhanced_before,
    enhanced_after,
    titles=(
        "Before: August 15, 2025 (S1C)",
        "After: August 16, 2025 (S1A)",
        "Changes"
    ),
    figsize=(18, 6)
)

plt.suptitle('Sentinel-1 SAR - Hainan Naval Base', fontsize=16, fontweight='bold', y=1.02)
plt.show()

print("\n💡 SAR Interpretation Tips:")
print("  • Bright pixels = Strong radar return (metal, buildings, ships)")
print("  • Dark pixels = Weak return (water, smooth surfaces)")
print("  • Medium gray = Moderate return (concrete, soil, vegetation)")

## 6. Change Detection Analysis

We'll use the **difference method** to detect changes between the two dates.

In [ ]:
# Compute change difference
print("Computing change difference...")
change_diff = compute_difference(enhanced_before, enhanced_after)

print(f"Change range: {change_diff.min():.2f} to {change_diff.max():.2f} dB")
print(f"Mean change: {change_diff.mean():.4f} dB")
print(f"Std change: {change_diff.std():.4f} dB")

# Detect significant changes (threshold = 3 dB)
print("\nDetecting significant changes (threshold = 3 dB)...")
change_map = detect_changes(enhanced_before, enhanced_after, threshold=3.0, method='difference')

# Refine change map (remove small regions)
refined_change_map = refine_change_map(change_map, min_size=20)

total_changed = np.sum(refined_change_map)
percent_changed = 100 * total_changed / refined_change_map.size

print(f"\n📊 Change Statistics:")
print(f"  Total changed pixels: {total_changed:,}")
print(f"  Percentage of area changed: {percent_changed:.2f}%")
print(f"  Approximate changed area: ~{total_changed * 100 / 10000:.2f} hectares")

In [ ]:
# Visualize change detection results
fig, axes = plot_before_after(
    enhanced_before,
    enhanced_after,
    change_map=refined_change_map,
    titles=(
        "Before (Aug 15)",
        "After (Aug 16)",
        "Detected Changes"
    ),
    figsize=(20, 6)
)

plt.suptitle('Change Detection Results', fontsize=16, fontweight='bold', y=1.02)
plt.show()

## 7. Change Magnitude Heatmap

In [ ]:
# Compute change magnitude
change_magnitude = compute_change_magnitude(enhanced_before, enhanced_after)

# Plot heatmap
fig, ax = plot_change_heatmap(
    change_magnitude,
    title="Change Magnitude Heatmap (Absolute Difference in dB)",
    cmap='RdYlGn_r',
    figsize=(14, 10)
)

plt.show()

print("\n🔥 Hottest Change Areas:")
top_changes = np.argsort(change_magnitude.flatten())[-10:][::-1]
for i, idx in enumerate(top_changes[:5], 1):
    y, x = np.unravel_index(idx, change_magnitude.shape)
    print(f"  {i}. Position ({y}, {x}): {change_magnitude[y, x]:.2f} dB")

## 8. Multi-Temporal RGB Composite

Create a false-color composite where:
- **Red** = After image (new bright targets)
- **Green/Blue** = Before image
- **Red areas** = Increases (new objects/activity)
- **Cyan areas** = Decreases (removed objects)

In [ ]:
fig, ax = plot_change_rgb(
    enhanced_before,
    enhanced_after,
    title="Multi-Temporal RGB Composite (Change Visualization)",
    figsize=(14, 12)
)

plt.show()

print("\n🎨 Color Interpretation:")
print("  • RED areas = Increased backscatter (new bright targets)")
print("  • CYAN areas = Decreased backscatter (removed targets)")
print("  • GRAY areas = No significant change")

## 9. Object Detection - Ships and Planes

Detect bright targets that could be ships (in port areas) or aircraft (on runway).

In [ ]:
# Detect bright spots in both images
print("Detecting bright targets (ships, planes, buildings)...\n")

# Before image
detections_before = detect_bright_spots(
    enhanced_before,
    percentile=99.5  # Top 0.5% brightest pixels
)

# Filter by size (remove noise, keep objects 5-500 pixels)
detections_before = filter_detections_by_size(detections_before, min_area=5, max_area=500)

print(f"Before (Aug 15): {len(detections_before)} objects detected")

# After image
detections_after = detect_bright_spots(
    enhanced_after,
    percentile=99.5
)

detections_after = filter_detections_by_size(detections_after, min_area=5, max_area=500)

print(f"After (Aug 16): {len(detections_after)} objects detected")
print(f"\nNet change: {len(detections_after) - len(detections_before):+d} objects")

In [ ]:
# Visualize detections - Before
fig, ax = plot_detections(
    enhanced_before,
    detections_before,
    title="Detected Objects - August 15, 2025",
    max_detections=30,
    figsize=(14, 12)
)

plt.show()

print("\nTop 10 Brightest Targets (Before):")
for i, det in enumerate(detections_before[:10], 1):
    print(f"  {i}. Area: {det['area']} px | Intensity: {det['intensity_max']:.2f} | "
          f"Location: ({det['centroid'][0]:.0f}, {det['centroid'][1]:.0f})")

In [ ]:
# Visualize detections - After
fig, ax = plot_detections(
    enhanced_after,
    detections_after,
    title="Detected Objects - August 16, 2025",
    max_detections=30,
    figsize=(14, 12)
)

plt.show()

print("\nTop 10 Brightest Targets (After):")
for i, det in enumerate(detections_after[:10], 1):
    print(f"  {i}. Area: {det['area']} px | Intensity: {det['intensity_max']:.2f} | "
          f"Location: ({det['centroid'][0]:.0f}, {det['centroid'][1]:.0f})")

## 10. Statistical Analysis

In [ ]:
# Compare detection statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Detection count comparison
axes[0, 0].bar(['Before\n(Aug 15)', 'After\n(Aug 16)'], 
               [len(detections_before), len(detections_after)],
               color=['steelblue', 'coral'])
axes[0, 0].set_ylabel('Number of Objects', fontsize=12)
axes[0, 0].set_title('Detected Object Count', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

# Area distribution
areas_before = [d['area'] for d in detections_before]
areas_after = [d['area'] for d in detections_after]

axes[0, 1].hist([areas_before, areas_after], bins=20, label=['Before', 'After'], 
                color=['steelblue', 'coral'], alpha=0.7)
axes[0, 1].set_xlabel('Object Area (pixels)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Object Size Distribution', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Intensity distribution
intensity_before = [d['intensity_max'] for d in detections_before]
intensity_after = [d['intensity_max'] for d in detections_after]

axes[1, 0].hist([intensity_before, intensity_after], bins=20, 
                label=['Before', 'After'], color=['steelblue', 'coral'], alpha=0.7)
axes[1, 0].set_xlabel('Maximum Intensity', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Object Intensity Distribution', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Change magnitude histogram
axes[1, 1].hist(change_magnitude.flatten(), bins=50, color='green', alpha=0.7)
axes[1, 1].axvline(3.0, color='red', linestyle='--', linewidth=2, label='Detection Threshold')
axes[1, 1].set_xlabel('Change Magnitude (dB)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Change Magnitude Distribution', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.suptitle('Statistical Analysis Summary', fontsize=16, fontweight='bold', y=1.01)
plt.show()

# Print summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)
print(f"\nDetected Objects:")
print(f"  Before: {len(detections_before)} objects")
print(f"  After: {len(detections_after)} objects")
print(f"  Change: {len(detections_after) - len(detections_before):+d} objects")

if areas_before:
    print(f"\nObject Sizes (Before):")
    print(f"  Mean: {np.mean(areas_before):.1f} px")
    print(f"  Median: {np.median(areas_before):.1f} px")
    print(f"  Range: {np.min(areas_before)}-{np.max(areas_before)} px")

if areas_after:
    print(f"\nObject Sizes (After):")
    print(f"  Mean: {np.mean(areas_after):.1f} px")
    print(f"  Median: {np.median(areas_after):.1f} px")
    print(f"  Range: {np.min(areas_after)}-{np.max(areas_after)} px")

print(f"\nChange Detection:")
print(f"  Total changed pixels: {np.sum(refined_change_map):,}")
print(f"  Percentage changed: {percent_changed:.2f}%")
print(f"  Mean change magnitude: {np.mean(change_magnitude):.4f} dB")
print(f"  Max change magnitude: {np.max(change_magnitude):.2f} dB")

## 11. Key Findings & Conclusions

In [ ]:
print("="*70)
print("KEY FINDINGS - HAINAN NAVAL BASE CHANGE ANALYSIS")
print("="*70)

print("\n📅 ANALYSIS PERIOD")
print(f"  • Before: August 15, 2025 (Sentinel-1C)")
print(f"  • After: August 16, 2025 (Sentinel-1A)")
print(f"  • Time difference: ~1 day")

print("\n📊 DATA CHARACTERISTICS")
print(f"  • Analyzed region: {roi_before.shape[0]} × {roi_before.shape[1]} pixels")
print(f"  • Ground coverage: ~{roi_before.shape[0]*10/1000:.1f} × {roi_before.shape[1]*10/1000:.1f} km²")
print(f"  • Spatial resolution: ~10m (Sentinel-1 GRD)")
print(f"  • Polarization: VV (vertical transmit/receive)")

print("\n🔍 CHANGE DETECTION RESULTS")
print(f"  • Changed pixels: {np.sum(refined_change_map):,} ({percent_changed:.2f}%)")
print(f"  • Detection threshold: 3.0 dB")
print(f"  • Method: Image differencing with morphological refinement")

print("\n🎯 OBJECT DETECTION RESULTS")
print(f"  • Objects before: {len(detections_before)}")
print(f"  • Objects after: {len(detections_after)}")
print(f"  • Net change: {len(detections_after) - len(detections_before):+d} objects")
print(f"  • Detection method: Bright spot analysis (top 0.5%)")

print("\n💡 INTERPRETATION")
print("  The 1-day time difference shows:")
if len(detections_after) > len(detections_before):
    print(f"    ✓ Increased activity: +{len(detections_after) - len(detections_before)} new bright targets")
    print("      (Possible ships, aircraft, or mobile equipment)")
elif len(detections_after) < len(detections_before):
    print(f"    ✓ Decreased activity: {len(detections_after) - len(detections_before)} fewer bright targets")
    print("      (Departure of ships, aircraft, or equipment movement)")
else:
    print("    ✓ Stable activity: Similar number of targets detected")

print(f"\n    • Overall change magnitude: {np.mean(change_magnitude):.4f} dB (mean)")
print(f"    • Changed area covers {percent_changed:.2f}% of analyzed region")

print("\n🔬 TECHNIQUES DEMONSTRATED")
print("  ✓ SAR image preprocessing (dB conversion, speckle filtering)")
print("  ✓ Change detection (difference method with thresholding)")
print("  ✓ Object detection (connected component analysis)")
print("  ✓ Multi-temporal visualization (RGB composite)")
print("  ✓ Statistical analysis and reporting")

print("\n" + "="*70)
print("Analysis complete! 🎉")
print("="*70)

## 12. Export Results (Optional)

Save high-resolution figures for reports and presentations.

In [ ]:
# Create output directory
output_dir = Path("../outputs/change_maps")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Exporting results to: {output_dir}\n")

# Export before/after comparison
fig, axes = plot_before_after(
    enhanced_before, enhanced_after, change_map=refined_change_map,
    titles=("Before (Aug 15)", "After (Aug 16)", "Changes"),
    figsize=(20, 6),
    save_path=output_dir / "01_before_after_comparison.png",
    dpi=300
)
plt.close()

# Export change heatmap
fig, ax = plot_change_heatmap(
    change_magnitude,
    title="Change Magnitude Heatmap",
    figsize=(14, 10),
    save_path=output_dir / "02_change_heatmap.png",
    dpi=300
)
plt.close()

# Export RGB composite
fig, ax = plot_change_rgb(
    enhanced_before, enhanced_after,
    title="Multi-Temporal RGB Composite",
    figsize=(14, 12),
    save_path=output_dir / "03_rgb_composite.png",
    dpi=300
)
plt.close()

# Export detections
fig, ax = plot_detections(
    enhanced_after, detections_after,
    title="Detected Objects - Aug 16, 2025",
    max_detections=30,
    figsize=(14, 12),
    save_path=output_dir / "04_detections.png",
    dpi=300
)
plt.close()

print("\n✓ All results exported successfully!")
print(f"  • Location: {output_dir.absolute()}")
print(f"  • Resolution: 300 DPI (publication quality)")
print(f"  • Files: 4 PNG images")

---

## Summary

This notebook demonstrated a complete SAR change detection workflow:

1. ✅ **Data Loading**: Multi-temporal Sentinel-1 SAR imagery
2. ✅ **Preprocessing**: dB conversion, speckle filtering, contrast enhancement
3. ✅ **Change Detection**: Difference method with thresholding
4. ✅ **Object Detection**: Bright spot analysis for ships/planes/infrastructure
5. ✅ **Visualization**: Before/after, heatmaps, RGB composites, annotated detections
6. ✅ **Statistical Analysis**: Quantitative assessment of changes
7. ✅ **Export**: High-resolution figures for reports

**Technologies Used:**
- Sentinel-1 C-band SAR
- Python (numpy, scipy, scikit-image, matplotlib)
- Computer Vision techniques
- Test-Driven Development

**Portfolio Value:**
- Demonstrates end-to-end remote sensing workflow
- Shows proficiency in SAR image analysis
- Professional code quality and documentation
- Reproducible research with clear methodology

---

*Analysis completed: 2025-10-01*  
*Notebook: 02_change_detection_analysis.ipynb*  
*Project: Sentinel SAR Change Detection - Hainan Naval Base*